### Задание 1
Для любых пар изображений которые имеют общее поле зрения (две фотографии сделанные с телефона который был повернут на определенный угол - любой лишь бы было 20+% перекрытия)
- Рассчитать матрицу афинного преобразования из одной системы координат в другую.
- Выполнить разложение данной матрицы и посчитать явно угол поворота, вектор переноса и скалирования. 
- Выполнить обратное преобразование - получить афинную матрицу заново. 

### Анализ
Для наших эксперементов подготовим сцену в программе Blender. Камера всегда будет направлена в надир.
Подготовим 2 кадра местности. 2 кадр будет повернут на 30 градусов и немного приближен. Качество найденной матрицы изучим визуальным путем наложив 1 изображение на 2 используя матрицу гомографии.

![Сцена в Blender](./assets/section1/blender.png)
- [Основной кадр](./assets/section1/1.png)
- [Кадр для сравнения](./assets/section1/2.png)

Код:

In [1]:
import cv2
import numpy as np

def extract_features(image, detector='sift', mask=None):
    if detector == 'sift':
        det = cv2.SIFT_create()
    elif detector == 'orb':
        det = cv2.ORB_create()
    elif detector == 'kaze':
        det = cv2.KAZE_create()
    elif detector == 'akaze':
        det = cv2.AKAZE_create()
    elif detector == 'brisk':
        det = cv2.BRISK_create()
        
    kp, des = det.detectAndCompute(image, mask)
    
    return kp, des

def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    if matching == 'BF':
        matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        if detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    return matches

def filter_matches_distance(matches, dist_threshold):
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold*n.distance:
            filtered_match.append(m)

    return filtered_match

def decompose_affine_matrix(affine):
    if affine.shape != (3, 3) or affine.dtype != np.float64:
        raise ValueError("Invalid input matrix. Must be a 3x3 double matrix.")

    R = affine[:2, :2]
    U, W, Vt = np.linalg.svd(R)

    rotation = np.dot(U, Vt)
    scaling = np.diag(W)
    translation = affine[:2, 2:]

    return rotation, translation, scaling

def recompose_affine_matrix(rotation, translation, scaling):
    rotation_scaling = np.dot(rotation, scaling)
    
    affine = np.eye(3)
    affine[:2, :2] = rotation_scaling
    affine[:2, 2] = translation.flatten()
    
    return affine

def get_angle(rotation):
    return np.arctan2(rotation[1, 0], rotation[0, 0])

def get_scale(scaling):
    return np.sqrt(scaling[0, 0] ** 2 + scaling[1, 1] ** 2)

def get_translation(translation):
    return translation

def get_angle_degrees(rotation):
    return np.degrees(get_angle(rotation))

img0 = cv2.imread("./assets/section1/1.png")
img1 = cv2.imread("./assets/section1/2.png")

kp0, des0 = extract_features(img0, 'sift')
kp1, des1 = extract_features(img1, 'sift')

matches = match_features(des0, des1, matching='BF', detector='sift', sort=True)
print('Number of matches before filtering:', len(matches))
matches = filter_matches_distance(matches, 0.75)
print('Number of matches after filtering:', len(matches))

src_pts = np.float32([kp0[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp1[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

H, _ = cv2.findHomography(src_pts, dst_pts, cv2.LMEDS)

print(f"Матрица гомографии: \n{H}")

rotation, translation, scaling = decompose_affine_matrix(H)

print(f"Угол: {get_angle_degrees(rotation)}")
print(f"Коэффициент увеличения: {get_scale(scaling)}")
print(f"Перевод: {get_translation(translation)}")

recomposed_affine = recompose_affine_matrix(rotation, translation, scaling)

print(f"Перекомпонованная матрица: \n{recomposed_affine}")

is_close = np.allclose(H, recomposed_affine, rtol=1e-04)
print(f"\nРавны ли матрица гомографии и перекомпонованная матрица? {is_close}")

# Наложим 1 изображение на 2 для проверки правильности найденной матрицы гомографии
height, width, _ = img0.shape
warped_image1 = cv2.warpPerspective(img1, np.linalg.inv(H), (width, height))
combined_image = cv2.addWeighted(warped_image1, 0.5, img0, 0.5, 0)
cv2.imwrite("./assets/section1/warped_image.png", combined_image)

Number of matches before filtering: 44680
Number of matches after filtering: 22785
Матрица гомографии: 
[[ 8.66020317e-01 -5.00002098e-01  5.31329760e+02]
 [ 5.00002399e-01  8.66034014e-01 -5.43454992e+02]
 [-3.09005308e-09  6.45127710e-09  1.00000000e+00]]
Угол: 30.000061086982882
Коэффициент увеличения: 1.414217309710615
Перевод: [[ 531.32975972]
 [-543.45499202]]
Перекомпонованная матрица: 
[[ 8.66033098e-01 -4.99998823e-01  5.31329760e+02]
 [ 5.00005673e-01  8.66021233e-01 -5.43454992e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Равны ли матрица гомографии и перекомпонованная матрица? True


True

Наложим одно изображение на другое:


![Сцена в Blender](./assets/section1/warped_image.png)

### Выводы 1 задания
- Матрица гомографии найдена достоточно точно. Это можно проверить визуально по наложенной картинке и по найденному углу 30.00006 (задан 30)
- Удалось получить начальную матрицу гомографии после ее декомпозиции (с учетом погрешности вычислений)
- Изучиил базовые принципы поиска и использования ключевых точек для поиска матрицы гомографии, сопоставления картинок.

### Задание 2
- Взять любую видеопоследовательность (желательно вид сверху) - например полет дрона. Можете скачать из  интернета или взять что то с работы. 
- Зафиксировав точку (например центр экрана) выполнить стабилизацию данной точки с использованием матрицы афинного преобразования 
(считая матрицу между кадрами вы всегда сможете понять где именно находится точка на изображении, а по отклонению самой точки вы можете определить точность самого алгоритма).
- Не берите 4к - долго считать, HD будет достаточно
- Посчитать расстояние между точками и ошибку в пикселях.
- Попробуйте добавить шум к изображению, посмотрите как меняется точность (положение точки). 
- Добавьте фильтр к изображению для улучшения контрастности (любой который найдете).
- Оцените как влияет выбор метода определения матрицы афинного преобразования - например estimateAffine2D и его аналоги (аналоги найдите самии).
- Оцените как влияет выбор детектора/дескриптора особых точек на результат. Какой метод даст лучший и самый быстрый результат? 

### Анализ
- Многие функции будем просто переиспользовать из 1 задания (требуется запускать ячейки последовательно).
- Отрендерим анимацию движения в 1 сторону и возврат в начальное положения. На видео добавим все варианты движения камеры (поворот, зум, перевод). В таком случае мы легко сможем найти ошибку т.к. мы точно знаем, что после преобразований точка должна вернуться к начальному положению.
- В качестве фильтра для улучшения контрастности будем использовать локальное повышение контрастности CLAHE.    
- Будем искать матрицу гомографии между 2 соседними кадрами видео и накапливать трансформации в H_accumulated.  
- Для визуального контроля правильности работы алгоритма будем строить карту. Для этого создадим холст размера которого нам хватит с запасом и используя матрицу гомографии и функцию warpPerspective будем налаживать по маске холст и каждый кадр.
- В качестве генератора шума будем использовать Гаусов шум

In [7]:
!ffmpeg -i ./assets/section2/0001-0480.mp4 -filter_complex "fps=12, scale=-1:360[s]; [s]split[a][b]; [a]palettegen[palette]; [b][palette]paletteuse" ./assets/section2/0001-0480_compressed.gif

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Тестовое видео (сжатое):

![Сцена в Blender](./assets/section2/0001-0480_compressed.gif )

In [ ]:
import math 
import time

def local_contrast_increase(frame):
    clahe = cv2.createCLAHE(clipLimit=5., tileGridSize=(8,8))
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab = cv2.merge((l2,a,b))
    
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def get_affine_matrix(matrix_type, src_pts, dst_pts, matrix_estimate_method):
    method = cv2.RANSAC

    if matrix_estimate_method == 'rho':
        method = cv2.RHO
    elif matrix_estimate_method == 'lmeds':
        method = cv2.LMEDS

    if matrix_type == 'findHomography':
        return cv2.findHomography(src_pts, dst_pts, method)
    elif matrix_type == 'estimateAffine2D':
        matrix, mask = cv2.estimateAffine2D(src_pts, dst_pts, method)
        return (np.vstack([matrix, [0, 0, 1]]), mask)
    elif matrix_type == 'estimateAffinePartial2D':
        matrix, mask = cv2.estimateAffinePartial2D(src_pts, dst_pts, method)
        return (np.vstack([matrix, [0, 0, 1]]), mask)

def add_gaussian_noise(image, mean=0, var=0.01):
    image = np.array(image, dtype=np.float32) / 255.0
    noise = np.random.normal(mean, var ** 0.5, image.shape)
    noisy_image = image + noise
    noisy_image = np.clip(noisy_image, 0, 1)
    noisy_image = np.uint8(noisy_image * 255)
    
    return noisy_image

def homography_experenent(use_local_contrast_filter, detector, matching, matrix_type, matrix_estimate_method, use_gaussian_noise, video_path = './assets/section2/0001-0480.mp4'):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    is_first_frame = True
    output_map_size = (frame_width // 2, frame_height)
    builded_map = np.zeros((output_map_size[1], output_map_size[0], 3), dtype=np.uint8) # Изменено для поддержки цветного изображения

    # Начальная матрица гомографии
    H_accumulated = np.array([[1, 0, frame_width // 8],
                            [0, 1, frame_height // 8],
                            [0, 0, 1]], dtype=np.float32)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.resize(frame, (frame_width // 4, frame_height // 4))
        
        if use_local_contrast_filter:
            frame = local_contrast_increase(frame)
        
        if use_gaussian_noise:
            frame = add_gaussian_noise(frame)

        if is_first_frame:
            is_first_frame = False
            kp0, des0 = extract_features(frame, detector)
            continue
        
        kp1, des1 = extract_features(frame, detector)
        
        try:
            matches = match_features(des0, des1, matching=matching, detector=detector, sort=True)
            matches = filter_matches_distance(matches, 0.75)
            src_pts = np.float32([kp0[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp1[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
            H, _ = get_affine_matrix(matrix_type, src_pts, dst_pts, matrix_estimate_method)
            H_accumulated = H_accumulated @ np.linalg.inv(H)
            kp0, des0 = kp1, des1
            warped_img = cv2.warpPerspective(frame, H_accumulated, output_map_size, borderMode=cv2.BORDER_TRANSPARENT) # BORDER_CONSTANT
            mask = np.zeros_like(builded_map, dtype=np.uint8)
            mask[warped_img > 0] = 1
            builded_map = cv2.add(builded_map * (1 - mask), warped_img * mask)
        except:
            return float('inf')
    
    if not use_gaussian_noise:
        cv2.imwrite(f"./assets/section2/builded_maps/builded_map_{detector}_{matching}_filtered-{use_local_contrast_filter}_{matrix_type}_{matrix_estimate_method}.png", builded_map)
    
    cap.release()
    
    return math.sqrt((H_accumulated[0, 2] - (frame_width // 8)) ** 2 + (H_accumulated[1, 2] - (frame_height // 8)) ** 2)

Подбор лучших гиперпараметров:

In [3]:
results = []

for detector in ['sift', 'orb', 'kaze', 'akaze', 'brisk']:
    for matching in ['BF', 'FLANN']: 
        for matrix_type in ['estimateAffine2D','findHomography', 'estimateAffinePartial2D']:
            for matrix_estimate_method in ['ransac', 'rho', 'lmeds']:
                for use_local_contrast_filter in [True, False]:
                    start_time = time.time()
                    error = homography_experenent(use_local_contrast_filter, detector, matching, matrix_type, matrix_estimate_method, False)
                    results.append((error, use_local_contrast_filter, detector, matching, matrix_type, matrix_estimate_method, time.time() - start_time))
results.sort(key=lambda x: x[0])

print('Параметры 20 лучших вариантов:')
for result in results[:20]:
    print(f"Ошибка: {result[0]:.4f} px, Params: {result[1:]}")

Параметры 20 лучших вариантов:
Ошибка: 0.2765 px, Params: (True, 'sift', 'BF', 'estimateAffinePartial2D', 'ransac', 40.382333278656006)
Ошибка: 0.2765 px, Params: (True, 'sift', 'BF', 'estimateAffinePartial2D', 'rho', 40.39733123779297)
Ошибка: 0.2765 px, Params: (True, 'sift', 'BF', 'estimateAffinePartial2D', 'lmeds', 39.792399644851685)
Ошибка: 0.3489 px, Params: (True, 'kaze', 'FLANN', 'estimateAffine2D', 'lmeds', 70.56719732284546)
Ошибка: 0.3565 px, Params: (True, 'kaze', 'FLANN', 'estimateAffine2D', 'rho', 69.8750958442688)
Ошибка: 0.4419 px, Params: (False, 'sift', 'FLANN', 'estimateAffinePartial2D', 'lmeds', 45.66717267036438)
Ошибка: 0.4613 px, Params: (True, 'sift', 'FLANN', 'estimateAffinePartial2D', 'ransac', 63.11133646965027)
Ошибка: 0.4898 px, Params: (False, 'kaze', 'FLANN', 'estimateAffine2D', 'ransac', 46.34456396102905)
Ошибка: 0.5215 px, Params: (False, 'sift', 'FLANN', 'estimateAffinePartial2D', 'ransac', 45.411500692367554)
Ошибка: 0.5504 px, Params: (True, 'sift'

Влияние шума на качество построения карты. Используем 20 лучших подобранных параметров и посмотрим на сколько изменилась ошибка.

In [5]:
best_results = results[:20]

for result in best_results:
    error, use_local_contrast_filter, detector, matching, matrix_type, matrix_estimate_method, _ = result
    noised_error = homography_experenent(use_local_contrast_filter, detector, matching, matrix_type, matrix_estimate_method, True)
    print(f"Параметры: {result[1:-1]}. Изменение ошибки: {noised_error - error} px")


Параметры: (True, 'sift', 'BF', 'estimateAffinePartial2D', 'ransac'). Изменение ошибки: 0.464865668529866 px
Параметры: (True, 'sift', 'BF', 'estimateAffinePartial2D', 'rho'). Изменение ошибки: 2.0499276667798023 px
Параметры: (True, 'sift', 'BF', 'estimateAffinePartial2D', 'lmeds'). Изменение ошибки: 0.9856152242130536 px
Параметры: (True, 'kaze', 'FLANN', 'estimateAffine2D', 'lmeds'). Изменение ошибки: 3.580521554536231 px
Параметры: (True, 'kaze', 'FLANN', 'estimateAffine2D', 'rho'). Изменение ошибки: 4.577513129107985 px
Параметры: (False, 'sift', 'FLANN', 'estimateAffinePartial2D', 'lmeds'). Изменение ошибки: 4.4092174867609355 px
Параметры: (True, 'sift', 'FLANN', 'estimateAffinePartial2D', 'ransac'). Изменение ошибки: -0.15477936254897773 px
Параметры: (False, 'kaze', 'FLANN', 'estimateAffine2D', 'ransac'). Изменение ошибки: 4.342403125137536 px
Параметры: (False, 'sift', 'FLANN', 'estimateAffinePartial2D', 'ransac'). Изменение ошибки: 5.618082757992347 px
Параметры: (True, 'sif

Пример хорошо построенной карты:  

![Пример хорошо построенной карты](./assets/section2/builded_maps/builded_map_sift_BF_filtered-False_estimateAffinePartial2D_ransac.png)  

Пример построенной карты с применением повышения контрастности:  

![Пример построенной карты с применением повышения контрастности](./assets/section2/builded_maps/builded_map_sift_BF_filtered-True_estimateAffinePartial2D_ransac.png)

Пример неудачно построенной карты:  

![Пример неудачно построенной карты](./assets/section2/builded_maps/builded_map_orb_BF_filtered-False_findHomography_ransac.png)  

[Посмотреть все сгенерированные карты](./assets/section2/builded_maps/).

### Выводы по 2 заданию
- На мой взгляд лучший алгоритм по соотношению качество / скорость это AKAZE т.к. он достаточно быстрый (быстрее SIFT и KAZE), дал приемлемую ошибку и визуально построенная карта выглядит с относительно малым количеством артефактов. Худшим алгоритмом оказался ORB. Хотя он самый быстрый, но в большинстве случаев давал максимальную ошибку и больше всего артефактов при построении карты. Алгоритмы SIFT и KAZE показывают минимальную ошибку, однако вычислительно затратны (есть реализации на GPU)
- Связка из методов поиска матрицы гомографии estimateAffine2D + lmeds дала меньше всего визуальных артефактов при построении карты
- При добавлении шума результаты ухудшились, что ожидаемо.
- 5 лучших результатов получены с использованием фильтра контрастности. Однако для этого видео его можно не использовать т.к. много контрастных элементов и хороших ключевых точек достаточно. 
- Для более точных результатов и автоматическому учету дисторсии камеры требуется взять инлайнеры например после RANSAC, построить более сложную модель с учетом дисторсии и находить матрицы гомографии через Ceres Solver
- Не имеет смысла обрабатывать каждый кадр т.к. изменения в нем могут быть несущественными. Это замедляет обработку и увеличивает ошибку машинных вычислений. Для решения этой проблемы можно использовать более быстрый `cv2.calcOpticalFlowPyrLK` и задать трэшхолд например 10 пикселей.
- Реальные кадры могут быть смазаны. Отсеять их можно например таким фильтром `cv2.Laplacian(frame, cv2.CV_64F).var() < 200`
- Построили карты основываясь на движении камеры, оценили ошибку программно и визуально.

### Задание 3
LoFTR  - любой другой, сравнить с готовыми решениями.


### Анализ
Воспользуемся алгоритмом LoFTR через библиотеку kornia

In [4]:
! pip install kornia

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import cv2
import torch
import kornia as K
from kornia.feature import LoFTR
import numpy as np
import math

cap = cv2.VideoCapture('./assets/section2/0001-0480.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
is_first_frame = True
output_map_size = (1280, 1800)
builded_map = np.zeros((output_map_size[1], output_map_size[0]), dtype=np.uint8)
loftr = LoFTR(pretrained='outdoor')
H_accumulated = np.array([[1, 0, frame_width // 8],
                        [0, 1, frame_height // 8],
                        [0, 0, 1]], dtype=np.float32)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (640, 480)) / 255.0

    if is_first_frame:
        prev_frame = frame
        is_first_frame = False
        continue
    
    frame_tensor = torch.from_numpy(frame).unsqueeze(0).unsqueeze(0).float()
    prev_frame_tensor = torch.from_numpy(prev_frame).unsqueeze(0).unsqueeze(0).float()
    input_dict = {"image0": frame_tensor, "image1": prev_frame_tensor}
    with torch.no_grad():
        correspondences = loftr(input_dict)

    mkpts0 = torch.from_numpy(correspondences['keypoints0'].cpu().numpy()).reshape(1, -1, 2)
    mkpts1 = torch.from_numpy(correspondences['keypoints1'].cpu().numpy()).reshape(1, -1, 2)
    H = K.geometry.find_homography_dlt(mkpts0, mkpts1)
    H_accumulated = H_accumulated @ H.squeeze().numpy()
    warped_img = cv2.warpPerspective(frame * 255, H_accumulated, output_map_size, borderMode=cv2.BORDER_TRANSPARENT).astype(np.uint8)
    mask = np.zeros_like(builded_map, dtype=np.uint8)
    mask[warped_img > 0] = 1
    builded_map = cv2.add(builded_map * (1 - mask), warped_img * mask)
    prev_frame = frame

cv2.imwrite("./assets/section3/builded_map_LoFTR.png", builded_map)
error = math.sqrt((H_accumulated[0, 2] - (frame_width // 8)) ** 2 + (H_accumulated[1, 2] - (frame_height // 8)) ** 2)

print(f"Ошибка: {error} px")

cap.release()

Downloading: "http://cmp.felk.cvut.cz/~mishkdmy/models/loftr_outdoor.ckpt" to /home/itiv422/.cache/torch/hub/checkpoints/loftr_outdoor.ckpt
100%|██████████| 44.2M/44.2M [02:11<00:00, 352kB/s]


Ошибка: 61.5907910089599 px


Карта:

![Карта](./assets/section3/builded_map_LoFTR.png)

### Выводы:
- Запустили алгоритм, использующий нейронные сети.
- LoFTR на CPU гораздо медленнее, чем классические алгоритмы из второго задания.
- Для текущего датасета алгоритм показал очень плохой результат (ошибка около 60 пикселей хотя некоторые классические давали более точный результат)
- Нейронные сети могут сопоставлять, например, ИК и визуальный канал (d2-net), что для классических алгоритмов очень сложно достичь.
